<a href="https://colab.research.google.com/github/raulbs7/Machine-Learning-Techniques-Project/blob/master/NLP_Supervised_Project/NLP_Hatred_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. NLP PREPROCESSING

## 1.1 Imports

In [69]:
!pip install tweet-preprocessor
!pip install emoji

In [70]:
from google.colab import files
import pandas as pd
import nltk
import io
import re
import string
import emoji
import functools
import operator

import preprocessor as p
from collections import Counter
from nltk.tokenize import sent_tokenize, word_tokenize
try:
    import cPickle as pickle
except ImportError: 
    import pickle

import html
import emo_unicode

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download("popular")
nltk.download('vader_lexicon') #sentiment analysis
nltk.download('twython') #twitter

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

False

## 1.2 Preprocessing steps

To preprocess the data, first it is needed to catch the data from the csv files.

In [71]:
def upload_dataframes (index_fields):
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
    df = pd.read_csv(io.StringIO(uploaded[fn].decode('utf-8')), index_col = index_fields)
    return df

In [72]:
tweets = upload_dataframes([])

Saving labeled_data.csv to labeled_data.csv
User uploaded file "labeled_data.csv" with length 2546446 bytes


In [73]:
print(tweets.shape)
tweets.head()

(24783, 7)


,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


### 1.2.1 Removing mentions and other words
For this data, it is important to remove some concepts like mentions that they doesn't give us any additional information and which makes much noise.

In [74]:
def remove_mentions(text):
  p.set_options(p.OPT.MENTION)
  return p.clean(text)

In [75]:
tweets['tweet'] = tweets['tweet'].apply(remove_mentions)
tweets.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT : As a woman you shouldn't complain abo...
1,1,3,0,3,0,1,!!!!! RT : boy dats cold...tyga dwn bad for cu...
2,2,3,0,3,0,1,!!!!!!! RT Dawg!!!! RT : You ever fuck a bitch...
3,3,3,0,2,1,1,!!!!!!!!! RT : she look like a tranny
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT : The shit you hear about me ...


### 1.2.2 Removing URLs
There are some tweets that have URLs that can be removed in order to not have conflicts later.

In [76]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

In [77]:
tweets['tweet'] = tweets['tweet'].apply(remove_urls)
tweets.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT : As a woman you shouldn't complain abo...
1,1,3,0,3,0,1,!!!!! RT : boy dats cold...tyga dwn bad for cu...
2,2,3,0,3,0,1,!!!!!!! RT Dawg!!!! RT : You ever fuck a bitch...
3,3,3,0,2,1,1,!!!!!!!!! RT : she look like a tranny
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT : The shit you hear about me ...


### 1.2.3 Removing symbols

One important thing is to remove symbols which are noise and not provide any information to the analysis of the text. The symbols that they are going to be removed are the next ones: !"#$%&()*+,-./:;<=>?@[\\]^_{|}~`

It is important to converse in UNICODE form the possible emojis, that can appear in some tweets, to preserve them from the removal of those symbols.

In [78]:
SYMBOLS_TO_REMOVE = re.escape('!"#$%&()*+,-./:;<=>?@[\\]^_{|}~`')

def remove_symbols(text):
  text = html.unescape(text) #Converting emojis
  return re.sub(f'[{SYMBOLS_TO_REMOVE}]', '', text)

In [79]:
tweets['tweet'] = tweets['tweet'].apply(remove_symbols)
tweets.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,RT As a woman you shouldn't complain about c...
1,1,3,0,3,0,1,RT boy dats coldtyga dwn bad for cuffin dat ...
2,2,3,0,3,0,1,RT Dawg RT You ever fuck a bitch and she sta...
3,3,3,0,2,1,1,RT she look like a tranny
4,4,6,0,6,0,1,RT The shit you hear about me might be true ...


### 1.2.4 Removal of capital letters

Next step in preprocessing is to remove the capital letters.

In [80]:
def remove_capital_let(text):
  return text.lower()

In [81]:
tweets['tweet'] = tweets['tweet'].apply(remove_capital_let)
tweets.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,rt as a woman you shouldn't complain about c...
1,1,3,0,3,0,1,rt boy dats coldtyga dwn bad for cuffin dat ...
2,2,3,0,3,0,1,rt dawg rt you ever fuck a bitch and she sta...
3,3,3,0,2,1,1,rt she look like a tranny
4,4,6,0,6,0,1,rt the shit you hear about me might be true ...


### 1.2.5 Removal of contractions

Next, it can be optional to remove contractions to have a better tokenization.

In [82]:
def remove_contractions(text):
  text = re.sub(r'isn\'t', r'is not', text)
  text = re.sub(r'aren\'t', r'are not', text)
  text = re.sub(r'wasn\'t', r'was not', text)
  text = re.sub(r'hadn\'t', r'had not', text)
  text = re.sub(r'haven\'t', r'have not', text)
  text = re.sub(r'wasn\'t', r'was not', text)
  text = re.sub(r'weren\'t', r'were not', text)
  text = re.sub(r'can\'t', r'can not', text)
  text = re.sub(r'doesn\'t', r'does not', text)
  text = re.sub(r'couldn\'t', r'could not', text)
  text = re.sub(r'shouldn\'t', r'should not', text)
  text = re.sub(r'don\'t', r'do not', text)
  text = re.sub(r'didn\'t', r'did not', text)
  text = re.sub(r'mustn\'t', r'must not', text)
  text = re.sub(r'might\'ve', r'might not', text)
  text = re.sub(r'won\'t', r'will not', text)
  text = re.sub(r'wouldn\'t', r'would not', text)
  text = re.sub(r'what\'s', r'what is', text)
  text = re.sub(r'where\'s', r'where is', text)
  text = re.sub(r'who\'s', r'who is', text)
  text = re.sub(r'who\'ll', r'who will', text)
  text = re.sub(r'that\'s', r'that is', text)
  text = re.sub(r'there\'s', r'there is', text)
  text = re.sub(r'i\'ll', r'i will', text)
  text = re.sub(r'i\'m', r'i am', text)
  text = re.sub(r'i\'have', r'i have', text)
  text = re.sub(r'i\'d', r'i would', text)
  #text = re.sub(r'i\'d', r'i had', text)
  text = re.sub(r'you\'re', r'you are', text)
  text = re.sub(r'you\'ve', r'you have', text)
  text = re.sub(r'you\'ll', r'you will', text)
  text = re.sub(r'you\'d', r'you would', text)
  #text = re.sub(r'you\'d', r'you had', text)
  text = re.sub(r'he\'s', r'he is', text)
  text = re.sub(r'he\'ll', r'he will', text)
  text = re.sub(r'she\'s', r'she is', text)
  text = re.sub(r'she\'ll', r'she will', text)
  text = re.sub(r'it\'s', r'it is', text)
  text = re.sub(r'it\'ll', r'it will', text)
  text = re.sub(r'we\'re', r'we are', text)
  text = re.sub(r'we\'ve', r'we have', text)
  text = re.sub(r'we\'ll', r'we will', text)
  text = re.sub(r'we\'d', r'we would', text)
  #text = re.sub(r'we\'d', r'we had', text)
  text = re.sub(r'they\'re', r'they are', text)
  text = re.sub(r'they\'ve', r'they have', text)
  text = re.sub(r'they\'ll', r'they will', text)
  text = re.sub(r'they\'d', r'they would', text)
  #text = re.sub(r'they\'d', r'they had', text)

  return text

In [83]:
tweets['tweet'] = tweets['tweet'].apply(remove_contractions)
tweets.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,rt as a woman you should not complain about ...
1,1,3,0,3,0,1,rt boy dats coldtyga dwn bad for cuffin dat ...
2,2,3,0,3,0,1,rt dawg rt you ever fuck a bitch and she sta...
3,3,3,0,2,1,1,rt she look like a tranny
4,4,6,0,6,0,1,rt the shit you hear about me might be true ...


### 1.2.6 Removal of repeated words

In [84]:
def remove_repeated(text):
  return re.sub(r'\b(\w+)( \1\b)+', r'\1', text)

There is an example to see how the previous function works because its complicated to show the effect in the dataset of tweets.

In [85]:
print(remove_repeated("hello hello 1 1 how how are are you you ?"))

tweets['tweet'] = tweets['tweet'].apply(remove_repeated)
tweets.head()

hello 1 how are you ?


,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,rt as a woman you should not complain about ...
1,1,3,0,3,0,1,rt boy dats coldtyga dwn bad for cuffin dat ...
2,2,3,0,3,0,1,rt dawg rt you ever fuck a bitch and she sta...
3,3,3,0,2,1,1,rt she look like a tranny
4,4,6,0,6,0,1,rt the shit you hear about me might be true ...


### 1.2.7 Tokenization

In order to determinine the lemma of a word based on its intended meaning, the following step must be carried out:

In [86]:
# 1. Tokenization

def tokenization_words(raw):
  tokens = word_tokenize(raw)
  tokens
  return tokens

def tokenization_sent(raw):
  tokens = sent_tokenize(raw)
  tokens
  return tokens

In [87]:
tweets_words = tweets['tweet'].apply(tokenization_words)

tweets_words.head()

0    [rt, as, a, woman, you, should, not, complain,...
1    [rt, boy, dats, coldtyga, dwn, bad, for, cuffi...
2    [rt, dawg, rt, you, ever, fuck, a, bitch, and,...
3                     [rt, she, look, like, a, tranny]
4    [rt, the, shit, you, hear, about, me, might, b...
Name: tweet, dtype: object

In [88]:
tweets_sent = tweets['tweet'].apply(tokenization_sent)
tweets_sent.head()

0    [ rt  as a woman you should not complain about...
1    [ rt  boy dats coldtyga dwn bad for cuffin dat...
2    [ rt dawg rt  you ever fuck a bitch and she st...
3                        [ rt  she look like a tranny]
4    [ rt  the shit you hear about me might be true...
Name: tweet, dtype: object

### 1.2.8 Stopwords

The stop words have to be filtered out before lemmatization. Stop words refers to the most common words in a language as prepositions or determiners.

In [89]:
def stop_words(text):
  #STOPWORDS
  english_stops = set(stopwords.words('english'))
  clean_sent = [word for word in text if word not in english_stops]
  return clean_sent
  #STOPWORDS 

In [90]:
tweets_clean = tweets_words.apply(stop_words)
tweets_clean.head()

0    [rt, woman, complain, cleaning, house, man, al...
1    [rt, boy, dats, coldtyga, dwn, bad, cuffin, da...
2    [rt, dawg, rt, ever, fuck, bitch, start, cry, ...
3                             [rt, look, like, tranny]
4    [rt, shit, hear, might, true, might, faker, bi...
Name: tweet, dtype: object

### 1.2.9 Conversion of emojis

Due to the fact that tweets are being treated, there is a lot of probability to have some emojis in our data. These emojis are represented in our data as a code. Our goal, is to convert this emojis in tokenizable and meaningful words.

In [91]:
def convert_emojis(text):
  
  em_split_emoji = emoji.get_emoji_regexp().split(text)
  em_split_whitespace = [substr.split() for substr in em_split_emoji]
  em_split = functools.reduce(operator.concat, em_split_whitespace)

  text = html.unescape(em_split[0])
  
  if len(em_split) > 0:
    if em_split[0] in emo_unicode.UNICODE_EMO:
      text = re.sub(re.escape('{}'.format(em_split[0])), r'_'.join(emo_unicode.UNICODE_EMO[em_split[0]].replace(r",",r"").replace(r":",r"").split()), em_split[0])
    else:
      text = em_split[0]
  else:
    text = ''
  return text

def treat_emojis(array):
  for i in range(len(array)):
    array[i] = convert_emojis(array[i])
  return array

In [92]:
tweets_clean = tweets_clean.apply(treat_emojis)
tweets_clean.head()

0    [rt, woman, complain, cleaning, house, man, al...
1    [rt, boy, dats, coldtyga, dwn, bad, cuffin, da...
2    [rt, dawg, rt, ever, fuck, bitch, start, cry, ...
3                             [rt, look, like, tranny]
4    [rt, shit, hear, might, true, might, faker, bi...
Name: tweet, dtype: object

### 1.2.10 Correction of wrong words

After that, it is necessary to correct wrong words of the text, in order to have a better analysis and to create a text sequence more understandable for our NLP algorithms.

In [93]:
uploaded = files.upload()
file_name = "big.txt"
raw = uploaded[file_name].decode("utf-8")

Saving big.txt to big.txt


In [94]:
def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(raw))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [95]:
def treat_correction(array):
  for i in range(len(array)):
    array[i] = correction(array[i])
  return array

In [96]:
tweets_clean = tweets_clean.apply(treat_emojis)
tweets_clean.head()

0    [rt, woman, complain, cleaning, house, man, al...
1    [rt, boy, dats, coldtyga, dwn, bad, cuffin, da...
2    [rt, dawg, rt, ever, fuck, bitch, start, cry, ...
3                             [rt, look, like, tranny]
4    [rt, shit, hear, might, true, might, faker, bi...
Name: tweet, dtype: object

### 1.2.11 Lemmatization

In [97]:
def lemmatizer_words(text):
  lemmatizer = WordNetLemmatizer()
  return [lemmatizer.lemmatize(word) for word in text]

In [98]:
tweets_lemmatizer=tweets_clean.apply(lemmatizer_words)
tweets_lemmatizer.head()

0    [rt, woman, complain, cleaning, house, man, al...
1    [rt, boy, dat, coldtyga, dwn, bad, cuffin, dat...
2    [rt, dawg, rt, ever, fuck, bitch, start, cry, ...
3                             [rt, look, like, tranny]
4    [rt, shit, hear, might, true, might, faker, bi...
Name: tweet, dtype: object

# 2. VECTORIZATION

In [102]:
from sklearn.feature_extraction.text import TfidfVectorizer

def tfid_vectorizer(raw):
  vectorizer = TfidfVectorizer(min_df=1)
  return vectorizer.fit_transform(raw);

def tfidf_ngram__vectorizer(raw):
  vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df=1)
  return vectorizer.fit_transform(raw);

#X = vectorizer.fit_transform(sent_tokenize)
tweet_vect = tweets_clean.apply(tfid_vectorizer)

print(tweet_vect)
#print(tweet_vect.toarray())


0          (0, 5)\t1.0\n  (1, 8)\t1.0\n  (2, 2)\t1.0\n ...
1          (0, 10)\t1.0\n  (1, 2)\t1.0\n  (2, 6)\t1.0\n...
2          (0, 6)\t1.0\n  (1, 3)\t1.0\n  (2, 6)\t1.0\n ...
3          (0, 2)\t1.0\n  (1, 1)\t1.0\n  (2, 0)\t1.0\n ...
4          (0, 4)\t1.0\n  (1, 5)\t1.0\n  (2, 2)\t1.0\n ...
                               ...                        
24778      (1, 4)\t1.0\n  (2, 2)\t1.0\n  (4, 5)\t1.0\n ...
24779      (0, 4)\t1.0\n  (1, 1)\t1.0\n  (2, 7)\t1.0\n ...
24780      (0, 10)\t1.0\n  (1, 1)\t1.0\n  (2, 9)\t1.0\n...
24781      (0, 5)\t1.0\n  (1, 1)\t1.0\n  (2, 4)\t1.0\n ...
24782      (0, 9)\t1.0\n  (1, 6)\t1.0\n  (2, 5)\t1.0\n ...
Name: tweet, Length: 24783, dtype: object


In [103]:
tweet_vect = tweets_clean.apply(tfidf_ngram__vectorizer)
#tfidf = X.toarray()
print(tweet_vect)
print (tweet_vect.shape)

#vectorizer.get_feature_names()

# KEYwORDS
#import numpy

#keywords = [vectorizer.get_feature_names()[numpy.argmax(x)] for x in X.toarray()]
# keywords

0          (0, 5)\t1.0\n  (1, 8)\t1.0\n  (2, 2)\t1.0\n ...
1          (0, 10)\t1.0\n  (1, 2)\t1.0\n  (2, 6)\t1.0\n...
2          (0, 6)\t1.0\n  (1, 3)\t1.0\n  (2, 6)\t1.0\n ...
3          (0, 2)\t1.0\n  (1, 1)\t1.0\n  (2, 0)\t1.0\n ...
4          (0, 4)\t1.0\n  (1, 5)\t1.0\n  (2, 2)\t1.0\n ...
                               ...                        
24778      (1, 4)\t1.0\n  (2, 2)\t1.0\n  (4, 5)\t1.0\n ...
24779      (0, 4)\t1.0\n  (1, 1)\t1.0\n  (2, 7)\t1.0\n ...
24780      (0, 10)\t1.0\n  (1, 1)\t1.0\n  (2, 9)\t1.0\n...
24781      (0, 5)\t1.0\n  (1, 1)\t1.0\n  (2, 4)\t1.0\n ...
24782      (0, 9)\t1.0\n  (1, 6)\t1.0\n  (2, 5)\t1.0\n ...
Name: tweet, Length: 24783, dtype: object
(24783,)
